<a href="https://colab.research.google.com/github/DhruvDarda/Super-Resolution/blob/main/DeepFace_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install deepface
from PIL import Image
import io
import os
import torchvision.transforms as transforms
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
from deepface import DeepFace
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 3.1 MB/s 
     |████████████████████████████████| 87 kB 8.4 MB/s 
     |████████████████████████████████| 60.9 MB 1.5 MB/s 
     |████████████████████████████████| 2.3 MB 43.0 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=08dac2803b87597f3e4c34916781d9140ab12961f64199647192e3913b3a9052
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requi

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
            nn.PReLU(),
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
        )

    def forward(self, x):
        return x + self.conv_block(x)

In [ ]:
class GeneratorResNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, n_residual_blocks=16):
        super(GeneratorResNet, self).__init__()

        # First layer
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, 64, kernel_size=9, stride=1, padding=4), nn.PReLU())

        # Residual blocks
        res_blocks = []
        for _ in range(n_residual_blocks):
            res_blocks.append(ResidualBlock(64))
        self.res_blocks = nn.Sequential(*res_blocks)

        # Second conv layer post residual blocks
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64, 0.8))

        # Upsampling layers
        upsampling = []
        for out_features in range(2):
            upsampling += [
                # nn.Upsample(scale_factor=2),
                nn.Conv2d(64, 256, 3, 1, 1),
                nn.BatchNorm2d(256),
                nn.PixelShuffle(upscale_factor=2),
                nn.PReLU(),
            ]
        self.upsampling = nn.Sequential(*upsampling)

        # Final output layer
        self.conv3 = nn.Sequential(nn.Conv2d(64, out_channels, kernel_size=9, stride=1, padding=4), nn.Tanh())

    def forward(self, x):
        out1 = self.conv1(x)
        out = self.res_blocks(out1)
        out2 = self.conv2(out)
        out = torch.add(out1, out2)
        out = self.upsampling(out)
        out = self.conv3(out)
        return out

In [ ]:
def to_np(x):
    return x.data.cpu().numpy()

In [ ]:
slpath = '/content/drive/MyDrive/happy_images/saved_models/generator_256.pt'
%cd /content/drive/MyDrive/ExpW_faces/
device = 'cuda' if torch.cuda.is_available() else 'cpu'

/content/drive/MyDrive/ExpW_faces


In [ ]:
'''
import glob
path =  glob.glob('/content/drive/MyDrive/AffectNet_faces_64x64/*.jpg')
for i in path:
  print(i)
  if os.path.isfile(i):
    os.remove(i)
'''

"\nimport glob\npath =  glob.glob('/content/drive/MyDrive/AffectNet_faces_64x64/*.jpg')\nfor i in path:\n  print(i)\n  if os.path.isfile(i):\n    os.remove(i)\n"

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
#from google.colab import files
#from IPython.display import Image


df_dict = {'angry': [], 'surprise': [], 'sad':[],'neutral':[], 'happy':[], 'fear':[], 'disgust':[]}

generator = GeneratorResNet()
generator.load_state_dict(torch.load(slpath, map_location=torch.device(device)))

hr_height = 256
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def add_margin(pil_img):
    width, height = pil_img.size
    left = (64-width)//2
    top = (64-height)//2
    result = Image.new(pil_img.mode, (64, 64), (0,0,0))
    result.paste(pil_img, (left, top))
    return result

def retain_shape(pil_img):
    width, height = pil_img.size
    shape = max(width, height)
    left = (shape - width)//2
    top = (shape - height)//2
    result = Image.new(pil_img.mode, (shape, shape), (0,0,0))
    result.paste(pil_img, (left, top))
    return result

def crop_image(pil_img, height, width):
    left = (64-width)*2
    top = (64-height)*2
    right = 256-(64-width)*2
    bottom = 256 - (64-height)*2
    pil_img = pil_img.crop((left, top, right, bottom))
    return pil_img


lr_transform = transforms.Compose(
            [
                transforms.Resize((hr_height // 4, hr_height // 4), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ])

hr_transform = transforms.Compose(
            [
                transforms.Resize((hr_height, hr_height), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ])

gen_transform = transforms.ToPILImage()

for i in df_dict.keys():
    os.mkdir('test/' + i)
    for img in tqdm(os.listdir('/content/drive/MyDrive/ExpW_faces/'+str(i))[:3000]):
        #img = 'test3.jpg' #files.upload()
        real_image = Image.open(str(i)+'/'+img).convert('RGB')
        o_height, o_width = real_image.size
        if o_height >= 256 and o_width >= 256:
            gen_image_pil = real_image
        else:
            if o_height < 60 or o_width < 60:
                real_image = add_margin(real_image)
            if o_height != o_width:
                real_image = retain_shape(real_image)
            recon_image = hr_transform(real_image) 
            real_image = lr_transform(real_image)
            real_image = torch.unsqueeze(real_image, dim=0)
            real_image = Variable(real_image.type(torch.Tensor))

            gen_image = generator(real_image)
            #gen_image_pil = gen_transform(gen_image.squeeze())
            gen_imagenp = to_np(gen_image.squeeze())
            gen_imagenp = (((gen_imagenp - gen_imagenp.min()) * 255) / (gen_imagenp.max() - gen_imagenp.min())).transpose(1, 2, 0).astype(np.uint8)
            gen_image_pil = Image.fromarray(gen_imagenp)
            #gen_image_pil = crop_image(gen_image_pil, o_width, o_height)

        name = 'test/' + i + '/' + img
        gen_image_pil.save(name)
        #print(gen_image_pil.size)

100%|██████████| 3000/3000 [40:05<00:00,  1.25it/s]


In [ ]:
import pandas as pd

%cd /content/drive/MyDrive/ExpW_faces
df_dict = {'surprise': [], 'sad':[],'neutral':[], 'angry':[]}
for i in df_dict.keys():
    print(i)
    result_dict = {i:[]}
    for img in tqdm(os.listdir('/content/drive/MyDrive/ExpW_faces/'+i)[:3000]):
        emotions_sr = DeepFace.analyze(img_path = i + '/' + img, detector_backend = "mtcnn", actions = ['emotion'], enforce_detection = False)
        result_dict[i].append(emotions_sr['dominant_emotion'])
    pd.Series(result_dict).to_csv(i+'.csv')


/content/drive/MyDrive/ExpW_faces
surprise


  0%|          | 0/3000 [00:00<?, ?it/s]

facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5

  0%|          | 0.00/5.98M [00:00<?, ?B/s]
100%|██████████| 5.98M/5.98M [00:00<00:00, 51.2MB/s]
100%|██████████| 3000/3000 [32:22<00:00,  1.54it/s]


sad


100%|██████████| 3000/3000 [24:41<00:00,  2.02it/s]


neutral


100%|██████████| 3000/3000 [30:32<00:00,  1.64it/s]


angry


100%|██████████| 3000/3000 [25:19<00:00,  1.97it/s]


In [ ]:
import pandas as pd
import os
%cd /content/drive/MyDrive/ExpW_faces
result_dict = {'angry': 0, 'surprise': 0, 'sad': 0,'neutral': 0, 'happy':  0, 'fear':0, 'disgust': 0}
count = 0
total = 0

for i in result_dict.keys():
    df = pd.read_csv(i + '.csv')

    result = df.iloc[0][1]

    if i == 'fear':
        count += result.count(i)
        result_dict[i] = (result.count(i)/len(os.listdir(i)))*100.17
        total += len(os.listdir(i))
        print(len(os.listdir(i)))
    else:
        count += result.count(i)
        result_dict[i] = (result.count(i)/3000)*100.17
        total += 3000
print(result_dict)
print('Total acc: ', (count/total)*100.17)

/content/drive/MyDrive/ExpW_faces
{'angry': 40.63563, 'surprise': 20.234340000000003, 'sad': 33.32322, 'neutral': 31.52016, 'happy': 53.557559999999995, 'fear': 27.019539473684212, 'disgust': 0.5342399999999999}
Total acc:  29.802992026856902


In [ ]:
import pandas as pd
import os
%cd /content/drive/MyDrive/AffectNet_faces/
result_dict = {'anger': 0, 'surprise': 0, 'sad': 0,'neutral': 0, 'happy':  0, 'fear':0, 'disgust': 0}
count = 0
total = 0

for i in result_dict.keys():
    df = pd.read_csv('results/' + i + '.csv')

    result = df.iloc[0][1]

    if i == 'anger':
        count += result.count('angry')
        result_dict[i] = result.count('angry')/len(os.listdir('anger'))*100.17
        total += len(os.listdir('anger'))
    else:
        count += result.count(i)
        result_dict[i] = result.count(i)/len(os.listdir(i))*100.17
        total += len(os.listdir(i))
print(result_dict)
print('Total acc: ', (count/total)*100.17)

/content/drive/MyDrive/AffectNet_faces
{'anger': 31.27903243540407, 'surprise': 19.67958100558659, 'sad': 30.284632653061227, 'neutral': 49.22617692907249, 'happy': 80.29484241823589, 'fear': 31.603923766816145, 'disgust': 3.756375}
Total acc:  39.76849457555083


In [ ]:
'''
    fig, axes = plt.subplots(1, 3, figsize=(12,5))
    labels = ['Low-Resolution Image', 'Super-Resolution Image', 'High-Resolution Image']
    imgs = [to_np(real_image), to_np(gen_image), to_np(recon_image)]
    for i, (ax, img) in enumerate(zip(axes.flatten(), imgs)):
        ax.axis('off')
        #ax.set_adjustable('box-forced')
        # Scale to 0-255
        img = img.squeeze()
        img = (((img - img.min()) * 255) / (img.max() - img.min())).transpose(1, 2, 0).astype(np.uint8)
        ax.imshow(img, cmap=None, aspect='equal')
        ax.set_title(labels[i])
    plt.subplots_adjust(wspace=0, hspace=0)

    title = 'sr'
    fig.text(0.5, 0.04, title, ha='center')
    plt.show()
    '''